In [1]:
import numpy as np
import pandas as pd

from natasha import (NamesExtractor, MorphVocab)

In [2]:
calls = pd.read_csv("C://Users/н/Downloads/test_data.csv")
calls.head()

,dlg_id,line_n,role,text
0,0,0,manager,Алло
1,0,1,client,Алло здравствуйте
2,0,2,manager,Добрый день
3,0,3,client,Меня зовут ангелина компания диджитал бизнес з...
4,0,4,manager,Ага


In [3]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [4]:
#calls

In [5]:
dialogs = []

d_call = {'manager': [calls['text'][0]], 'client': []}
last_id = 0

for row_ind in range(1, calls.shape[0]) :
    
    if calls['line_n'][row_ind] == 0:
        dialogs.append(d_call)
        d_call = {'manager': [], 'client': []}
        d_call[calls['role'][row_ind]].append(calls['text'][row_ind])
    else:
        d_call[calls['role'][row_ind]].append(calls['text'][row_ind])

dialogs.append(d_call)    


In [6]:
managers_work = {}

main_tasks = {'greeted': False, 
              'introduced': False, 
              'manager_name': None, 
              'company_name': None, 
              'told_goodbye': False, 
              'greeted_and_told_goodbye': False
             }

In [7]:
morph_vocab = MorphVocab()
name_extractor = NamesExtractor(morph_vocab)

In [8]:
def if_greeted(manager_phrases):
    global dct
    greeting_phrases = ['обрый день', 'ень добрый', 'дравствуйте', 'ривет', 'обрый']
    
    for manager_phrase in manager_phrases:
        for greeting in greeting_phrases:
            if greeting in manager_phrase:
                dct['greeted'] = manager_phrase
                return True
    return False
                
def if_introduced(manager_phrases):
    global dct
    intro_phrases = ['еня зовут', 'ое имя']
    
    for manager_phrase in manager_phrases:
        for intro in intro_phrases:
            if intro in manager_phrase:
                dct['introduced'] = manager_phrase
                splitted = manager_phrase.split(' ')
                dct['manager_name'] = splitted[splitted.index('имя') + 1] if 'имя' in splitted else splitted[splitted.index('зовут') + 1]
                return True
    return False

def if_told_goodbye(manager_phrases):
    global dct
    goodbye_phrases = ['о свидания', 'сего доброго']
    
    for manager_phrase in manager_phrases:
        for goodbye in goodbye_phrases:
            if goodbye in manager_phrase:
                dct['told_goodbye'] = manager_phrase
                return True
    return False

def if_greeted_and_told_goodbye(beg_phrases, end_phrases):
    global dct
    dct['greeted_and_told_goodbye'] = (if_greeted(beg_phrases) and (if_told_goodbye(end_phrases)))

def find_name(dialog_ind):
    global dialogs
    possible_names, stop_names = [], []
    default = ['ое имя ', 'еня зовут ', 'я ']
    
    for sentence in (dialogs[dialog_ind]['client']):
        matches = name_extractor(sentence)
        for match in matches:
            if (match.fact.first != None):
                mf = str(match.fact.first)
                for greet in default:
                    if len(mf) > 3 and (greet + mf not in sentence) and ('еня ' + mf + ' зовут' not in sentence) and (mf + ' собачка' not in sentence):
                        possible_names.append(mf)
                    else:
                        stop_names.append(mf)
    
    return possible_names, stop_names

def find_orig_name(pos_names, stop_names):
    orig = set(pos_names).difference(set(stop_names)).difference({'полей'})
    dt = {}
    
    for name in orig:
        if name[-2:] not in ('ов', 'ев', 'ян', 'ин'):
            if pos_names.count(name) in dt.keys():
                dt[pos_names.count(name)].append(name)
            else:
                dt[pos_names.count(name)] = [name]
            
    for key in list(dt.keys())[::-1]:
        if len(dt[key]) == 1:
            return dt[key][0].capitalize()
        else:
            for n in pos_names[::-1]:
                if n in dt[key]:
                    return n.capitalize()
        
def find_company(dialog):
    for sentence in dialog:
        if 'компания' in sentence.lower():
            splt = sentence.split(' ')
            return splt[splt.index('компания') + 1].capitalize()
            

In [9]:
for call in range(len(dialogs)):
    area = 7 if len(dialogs[call]['manager']) > 7 else len(dialogs[call]['manager'])
    dct = main_tasks.copy()
    beginning = dialogs[call]['manager'][:area] 
    ending = dialogs[call]['manager'][-area:]
    
    if_greeted_and_told_goodbye(beginning, ending)
    
    if not if_introduced(beginning):
        p_names, s_names = find_name(call)
        dct['manager_name'] = find_orig_name(p_names, s_names)
    
    dct['company_name'] = find_company(dialogs[call]['client'])
            
         
    
    managers_work[str(call)] = dct

In [10]:
data = pd.DataFrame(managers_work).T

In [11]:
export_data = data.to_csv('all_calls.csv', index=None, header=True)